<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [3]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

stock_ids = stock_n100

mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AUBANK', 'AXISBANK', 'BAJAJFINSV', 'BAJFINANCE',
       'BANKBARODA', 'BATAINDIA', 'BERGEPAINT', 'BHARTIARTL', 'BRITANNIA',
       'CANBK', 'CIPLA', 'COLPAL', 'DABUR', 'DEEPAKNTR', 'DIVISLAB',
       'DMART', 'DRREDDY', 'EICHERMOT', 'EMAMILTD', 'FEDERALBNK',
       'GODREJCP', 'GODREJIND', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCBANK',
       'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO', 'HINDUNILVR', 'IBREALEST',
       'ICICIBANK', 'ICICIPRULI', 'INDIANB', 'INDIGO', 'INDUSINDBK',
       'INDUSTOWER', 'INFY', 'IRCTC', 'ITC', 'JUBLFOOD', 'KOTAKBANK',
       'LT', 'LTIM', 'M&M', 'MARUTI', 'MCDOWELL-N', 'MOTILALOFS',
       'MUTHOOTFIN', 'NAM-INDIA', 'NATIONALUM', 'NAUKRI', 'PFIZER',
       'PIDILITIND', 'PVR', 'RELIANCE', 'SAIL', 'SBILIFE', 'SBIN',
       'SUNPHARMA', 'TATACONSUM', 'TATAMOTORS', 'TATAPOWER', 'TATASTEEL',
       'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'VOLTAS', 'WHIRLPOOL',
       'WIPRO', 'ZEEL'], dtype=object)

In [4]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    60
1    13
Name: InPortfolio, dtype: int64

In [5]:
# consistent compounders
mypf[mypf['InPortfolio'] == 1]['CCIFlag'].value_counts()

0    8
1    5
Name: CCIFlag, dtype: int64

In [6]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [7]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"


  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [8]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='2y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    #stock_df.ta.macd(close='Close', fast=12, slow=26, append=True)
    #stock_df['Trend'] = 'Down'
    #stock_df.loc[stock_df['MACD_12_26_9'] - stock_df['MACDs_12_26_9'] > 10, 'Trend'] = 'Up'
    #stock_df.ta.eom(append=True)    
    stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'EOM_14_100000000':'EOM'}, inplace=True)
    stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0) 
    #stock_df['EOM'] = round(stock_df['EOM'],0) 
    #stock_df['EOM'] = '-ve'
    #stock_df.loc[stock_df['EOM_14_100000000'] > 0, 'EOM'] = '+ve'    
    stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df['Upside%52W'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   #'MACD_12_26_9','MACDh_12_26_9','MACDs_12_26_9','EOM_14_100000000'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]    
    tmp['Curr_PE'] = current_pe
    tmp['EstmtFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['EstmtFlag'].values[0] 
    tmp['DuraFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['DuraFlag'].values[0]         
    df_prec_dev = df_prec_dev.append(tmp)

In [9]:
# nifty in last 5 days
stock_prec_dev('^NSEI').tail(5)

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Upside%52W
Date,,,,,,,,,,,
2023-01-19 00:00:00+05:30,18108.0,^NSEI,4.75,0.19,-0.04,18812.0,15294.0,79.98,20.02,50.0,4.0
2023-01-20 00:00:00+05:30,18028.0,^NSEI,4.28,0.17,-0.07,18812.0,15294.0,77.70,22.30,47.0,4.0
2023-01-23 00:00:00+05:30,18119.0,^NSEI,4.80,0.15,-0.07,18812.0,15294.0,80.29,19.71,51.0,4.0
2023-01-24 00:00:00+05:30,18118.0,^NSEI,4.80,0.14,-0.08,18812.0,15294.0,80.28,19.72,51.0,4.0
2023-01-25 00:00:00+05:30,17892.0,^NSEI,3.48,0.13,-0.14,18812.0,15294.0,73.85,26.15,43.0,5.0


In [10]:
# Upside% Calculations
df_prec_dev['Upside%FC'] = round((df_prec_dev['Forecast']-df_prec_dev['Close'])*100/(df_prec_dev['Close']),0)
df_prec_dev['Upside%Avg'] = round((df_prec_dev['Upside%FC'] + df_prec_dev['Upside%52W'])/2,0)
#df_prec_dev['Upside%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Curr_PE']),0)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%FC', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
#df_prec_dev[df_prec_dev['Trend'] == 'Up'].sort_values(by = 'Upside%', ascending=False)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [11]:
# portfolio stocks
cols = ['Stock', 'Close', 'Dev%_200', 'Slope%_200', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Avg_Cost', 'Med_PE', 'Curr_PE', 'Upside%FC', 'Upside%52W', 'Upside%Avg', 'EstmtFlag','DuraFlag']
df_prec_dev = df_prec_dev[cols]
cols = ['Stock', 'Close', 'Dev%_200', 'Slope%_200', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Avg_Cost', 'Med_PE', 'Curr_PE', 'EstmtFlag','DuraFlag']

mypf['Stock'] = mypf['Symbol'] + '.NS'
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df['Profit%FC'] = round((df_prec_dev['Forecast']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
tmp_df['Profit%52W'] = round((df_prec_dev['Max']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
tmp_df['Profit%Avg'] = round((tmp_df['Profit%FC'] + tmp_df['Profit%52W'])/2,0)
print('qualified stocks: '+str(tmp_df.shape[0]))

cols = ['Stock', 'Close', 'Dev%_200', 'Slope%_200', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Avg_Cost', 'Med_PE', 'Curr_PE', 'Profit%FC', 'Profit%52W', 'Profit%Avg', 'EstmtFlag','DuraFlag']
tmp_df = tmp_df[cols]
tmp_df.sort_values(by = 'Profit%Avg', ascending=False)

qualified stocks: 13


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag
13,VOLTAS.NS,757.0,-19.86,-2.54,-0.49,1315.0,757.0,-0.08,100.08,27.0,976.0,821.00,43.4,142.0,19.0,60.0,40.0,0.0,1.0
12,JUBLFOOD.NS,494.0,-10.86,-0.36,-1.15,687.0,463.0,13.77,86.23,38.0,636.0,499.45,87.9,68.8,27.0,38.0,32.0,1.0,1.0
30,TATAMOTORS.NS,419.0,-1.40,-0.47,-0.49,518.0,372.0,32.23,67.77,58.0,491.0,392.00,19.2,1000.0,25.0,32.0,28.0,1.0,-1.0
7,MOTILALOFS.NS,692.0,-8.28,-1.19,0.43,937.0,654.0,13.59,86.41,42.0,915.0,733.00,19.7,9.6,25.0,28.0,26.0,-1.0,0.0
53,TECHM.NS,1051.0,-0.83,-1.98,0.00,1520.0,935.0,19.85,80.15,56.0,1124.0,1048.00,18.7,19.4,7.0,45.0,26.0,-1.0,1.0
28,MUTHOOTFIN.NS,1040.0,-4.23,-1.20,-0.37,1447.0,956.0,17.12,82.88,37.0,1227.0,1082.00,12.5,11.1,13.0,34.0,24.0,-1.0,0.0
10,BATAINDIA.NS,1530.0,-13.33,-1.02,-1.03,2020.0,1530.0,-0.04,100.04,28.0,1998.0,1632.00,62.3,63.5,22.0,24.0,23.0,1.0,1.0
15,ICICIPRULI.NS,464.0,-9.70,-0.39,-0.22,595.0,440.0,15.58,84.42,47.0,594.0,484.00,60.9,87.3,23.0,23.0,23.0,1.0,0.0
31,BERGEPAINT.NS,549.0,-11.35,-1.13,-0.68,739.0,549.0,-0.13,100.13,33.0,637.0,570.00,72.9,56.4,12.0,30.0,21.0,1.0,1.0
17,NAUKRI.NS,3668.0,-7.99,-1.08,-1.02,5096.0,3363.0,17.61,82.39,38.0,4684.0,4154.00,78.8,15.9,13.0,23.0,18.0,1.0,0.0


In [12]:
# wait for max profit

tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)].sort_values(by = 'Profit%Avg', ascending=False)

,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag
12,JUBLFOOD.NS,494.0,-10.86,-0.36,-1.15,687.0,463.0,13.77,86.23,38.0,636.0,499.45,87.9,68.8,27.0,38.0,32.0,1.0,1.0
10,BATAINDIA.NS,1530.0,-13.33,-1.02,-1.03,2020.0,1530.0,-0.04,100.04,28.0,1998.0,1632.00,62.3,63.5,22.0,24.0,23.0,1.0,1.0
31,BERGEPAINT.NS,549.0,-11.35,-1.13,-0.68,739.0,549.0,-0.13,100.13,33.0,637.0,570.00,72.9,56.4,12.0,30.0,21.0,1.0,1.0
22,HDFCLIFE.NS,585.0,4.29,0.53,1.03,642.0,502.0,59.32,40.68,46.0,709.0,578.00,89.3,83.1,23.0,11.0,17.0,1.0,1.0
38,INFY.NS,1543.0,2.73,-1.00,-0.19,1866.0,1351.0,37.28,62.72,55.0,1742.0,1628.00,23.0,27.2,7.0,15.0,11.0,1.0,1.0


In [13]:
# keep a watch
tmp_df[((tmp_df['EstmtFlag'] == 0) & (tmp_df['DuraFlag'] != -1)) | ((tmp_df['DuraFlag'] == 0) & (tmp_df['EstmtFlag'] != -1))].sort_values(by = 'Profit%Avg', ascending=False)

,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag
13,VOLTAS.NS,757.0,-19.86,-2.54,-0.49,1315.0,757.0,-0.08,100.08,27.0,976.0,821.0,43.4,142.0,19.0,60.0,40.0,0.0,1.0
15,ICICIPRULI.NS,464.0,-9.70,-0.39,-0.22,595.0,440.0,15.58,84.42,47.0,594.0,484.0,60.9,87.3,23.0,23.0,23.0,1.0,0.0
17,NAUKRI.NS,3668.0,-7.99,-1.08,-1.02,5096.0,3363.0,17.61,82.39,38.0,4684.0,4154.0,78.8,15.9,13.0,23.0,18.0,1.0,0.0


In [14]:
# sell as soon as you can
tmp_df[(tmp_df['EstmtFlag'] == -1) | (tmp_df['DuraFlag'] == -1)].sort_values(by = 'Profit%Avg', ascending=False)

,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag
30,TATAMOTORS.NS,419.0,-1.40,-0.47,-0.49,518.0,372.0,32.23,67.77,58.0,491.0,392.0,19.2,1000.0,25.0,32.0,28.0,1.0,-1.0
7,MOTILALOFS.NS,692.0,-8.28,-1.19,0.43,937.0,654.0,13.59,86.41,42.0,915.0,733.0,19.7,9.6,25.0,28.0,26.0,-1.0,0.0
53,TECHM.NS,1051.0,-0.83,-1.98,0.00,1520.0,935.0,19.85,80.15,56.0,1124.0,1048.0,18.7,19.4,7.0,45.0,26.0,-1.0,1.0
28,MUTHOOTFIN.NS,1040.0,-4.23,-1.20,-0.37,1447.0,956.0,17.12,82.88,37.0,1227.0,1082.0,12.5,11.1,13.0,34.0,24.0,-1.0,0.0
52,DIVISLAB.NS,3336.0,-8.94,-1.47,0.21,4530.0,3266.0,5.51,94.49,45.0,3558.0,3671.0,45.2,29.6,-3.0,23.0,10.0,-1.0,1.0


In [15]:
cols = ['Stock', 'Close', 'Dev%_200', 'Slope%_200', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Med_PE', 'Curr_PE','Upside%FC', 'Upside%52W', 'Upside%Avg','EstmtFlag','DuraFlag']
df_prec_dev = df_prec_dev[cols]

In [16]:
# prospect consistent compounders (CC)
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] != -1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Upside%Avg'] > 20]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

qualified stocks: 5


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag
12,JUBLFOOD.NS,494.0,-10.86,-0.36,-1.15,687.0,463.0,13.77,86.23,38.0,636.0,87.9,68.8,29.0,39.0,34.0,1.0,1.0
31,BERGEPAINT.NS,549.0,-11.35,-1.13,-0.68,739.0,549.0,-0.13,100.13,33.0,637.0,72.9,56.4,16.0,35.0,26.0,1.0,1.0
58,LTIM.NS,4431.0,-2.11,-2.03,-1.54,6390.0,3796.0,24.47,75.53,55.0,4663.0,26.6,37.9,5.0,44.0,24.0,1.0,0.0
20,TITAN.NS,2357.0,-2.90,-0.25,-1.11,2770.0,1929.0,50.84,49.16,30.0,2980.0,78.0,66.1,26.0,18.0,22.0,1.0,1.0
34,ASIANPAINT.NS,2775.0,-10.45,-0.32,-0.82,3566.0,2576.0,20.10,79.90,24.0,3180.0,73.4,69.7,15.0,29.0,22.0,1.0,1.0


In [17]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] != -1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Upside%Avg'] > 20]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

qualified stocks: 12


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag
0,GODREJIND.NS,424.0,-5.94,-0.44,0.45,622.0,405.0,8.85,91.15,37.0,638.0,39.5,18.50,50.0,47.0,48.0,0.0,1.0
2,BAJFINANCE.NS,5796.0,-12.04,-0.99,-1.73,7720.0,5265.0,21.63,78.37,26.0,7869.0,50.8,35.40,36.0,33.0,34.0,1.0,1.0
11,BAJAJFINSV.NS,1314.0,-12.14,-1.14,-2.41,1805.0,1093.0,31.09,68.91,18.0,1691.0,39.2,38.30,29.0,37.0,33.0,1.0,0.0
9,WHIRLPOOL.NS,1410.0,-12.11,-0.50,-0.67,1845.0,1410.0,-0.07,100.07,29.0,1854.0,72.2,69.00,31.0,31.0,31.0,1.0,0.0
48,NATIONALUM.NS,83.0,3.31,-2.50,1.28,126.0,66.0,27.75,72.25,56.0,90.0,10.3,5.98,8.0,52.0,30.0,0.0,1.0
32,TATAPOWER.NS,201.0,-10.47,-0.89,-0.93,287.0,195.0,7.01,92.99,35.0,232.0,20.4,25.30,15.0,43.0,29.0,1.0,0.0
6,PFIZER.NS,4011.0,-5.78,-0.23,-0.75,4599.0,4011.0,-0.01,100.01,24.0,5297.0,36.5,120.00,32.0,15.0,24.0,0.0,1.0
58,LTIM.NS,4431.0,-2.11,-2.03,-1.54,6390.0,3796.0,24.47,75.53,55.0,4663.0,26.6,37.90,5.0,44.0,24.0,1.0,0.0
20,TITAN.NS,2357.0,-2.90,-0.25,-1.11,2770.0,1929.0,50.84,49.16,30.0,2980.0,78.0,66.10,26.0,18.0,22.0,1.0,1.0
34,ASIANPAINT.NS,2775.0,-10.45,-0.32,-0.82,3566.0,2576.0,20.10,79.90,24.0,3180.0,73.4,69.70,15.0,29.0,22.0,1.0,1.0


In [18]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] != -1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Discount%'] > 87]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Premium%', ascending=True)

qualified stocks: 4


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag
9,WHIRLPOOL.NS,1410.0,-12.11,-0.50,-0.67,1845.0,1410.0,-0.07,100.07,29.0,1854.0,72.2,69.0,31.0,31.0,31.0,1.0,0.0
6,PFIZER.NS,4011.0,-5.78,-0.23,-0.75,4599.0,4011.0,-0.01,100.01,24.0,5297.0,36.5,120.0,32.0,15.0,24.0,0.0,1.0
32,TATAPOWER.NS,201.0,-10.47,-0.89,-0.93,287.0,195.0,7.01,92.99,35.0,232.0,20.4,25.3,15.0,43.0,29.0,1.0,0.0
0,GODREJIND.NS,424.0,-5.94,-0.44,0.45,622.0,405.0,8.85,91.15,37.0,638.0,39.5,18.5,50.0,47.0,48.0,0.0,1.0


In [19]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] != -1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 12


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag
68,INDIANB.NS,292.0,41.14,3.38,0.70,300.0,127.0,95.46,4.54,55.0,284.0,7.4,7.17,-3.0,3.0,0.0,1.0,1.0
64,SUNPHARMA.NS,1040.0,11.71,0.64,0.29,1052.0,791.0,95.42,4.58,59.0,1056.0,NaN,31.40,2.0,1.0,2.0,1.0,1.0
42,M&M.NS,1312.0,12.19,2.22,0.24,1354.0,684.0,93.66,6.34,56.0,1475.0,20.5,19.50,12.0,3.0,8.0,1.0,1.0
66,HCLTECH.NS,1123.0,12.95,-0.10,0.37,1148.0,857.0,91.32,8.68,67.0,1117.0,16.3,21.10,-1.0,2.0,0.0,1.0,1.0
47,HDFC.NS,2711.0,12.99,0.58,0.34,2775.0,2053.0,91.08,8.92,57.0,2940.0,23.0,20.50,8.0,2.0,5.0,1.0,0.0
72,BRITANNIA.NS,4388.0,16.43,1.67,0.55,4526.0,2995.0,91.00,9.00,56.0,3935.0,NaN,67.20,-10.0,3.0,-4.0,1.0,0.0
46,HDFCBANK.NS,1649.0,12.46,0.48,0.37,1696.0,1281.0,88.59,11.41,55.0,1813.0,26.0,21.00,10.0,3.0,6.0,NaN,NaN
60,GODREJCP.NS,914.0,7.81,1.06,1.01,948.0,665.0,88.06,11.94,53.0,958.0,NaN,57.80,5.0,4.0,4.0,NaN,NaN
70,LT.NS,2175.0,17.61,1.19,1.08,2273.0,1453.0,88.01,11.99,53.0,2036.0,NaN,31.80,-6.0,5.0,-0.0,NaN,NaN
49,HINDUNILVR.NS,2622.0,6.43,1.22,0.46,2722.0,1897.0,87.92,12.08,51.0,2832.0,NaN,62.30,8.0,4.0,6.0,1.0,1.0


In [20]:
# prospects turning around (exhaustive)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] != -1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > -.5) & (tmp_df['Slope%_200'] < 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 10


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag
65,TCS.NS,3430.0,6.95,-0.37,0.49,3735.0,2915.0,62.77,37.23,67.0,3385.0,28.1,30.8,-1.0,9.0,4.0,1.0,0.0
0,GODREJIND.NS,424.0,-5.94,-0.44,0.45,622.0,405.0,8.85,91.15,37.0,638.0,39.5,18.5,50.0,47.0,48.0,0.0,1.0
66,HCLTECH.NS,1123.0,12.95,-0.10,0.37,1148.0,857.0,91.32,8.68,67.0,1117.0,16.3,21.1,-1.0,2.0,0.0,1.0,1.0
27,TATACONSUM.NS,740.0,-4.63,-0.13,-0.38,849.0,655.0,43.84,56.16,32.0,871.0,NaN,66.4,18.0,15.0,16.0,1.0,1.0
25,RELIANCE.NS,2383.0,-6.60,-0.31,-0.66,2811.0,2229.0,26.38,73.62,30.0,2854.0,25.3,25.3,20.0,18.0,19.0,1.0,0.0
6,PFIZER.NS,4011.0,-5.78,-0.23,-0.75,4599.0,4011.0,-0.01,100.01,24.0,5297.0,36.5,120.0,32.0,15.0,24.0,0.0,1.0
33,MCDOWELL-N.NS,769.0,-8.29,-0.36,-0.79,942.0,732.0,17.40,82.60,19.0,885.0,74.5,47.0,15.0,22.0,18.0,0.0,1.0
34,ASIANPAINT.NS,2775.0,-10.45,-0.32,-0.82,3566.0,2576.0,20.10,79.90,24.0,3180.0,73.4,69.7,15.0,29.0,22.0,1.0,1.0
56,PIDILITIND.NS,2315.0,-6.98,-0.04,-1.04,2902.0,1991.0,35.60,64.40,27.0,2448.0,70.6,94.6,6.0,25.0,16.0,1.0,1.0
20,TITAN.NS,2357.0,-2.90,-0.25,-1.11,2770.0,1929.0,50.84,49.16,30.0,2980.0,78.0,66.1,26.0,18.0,22.0,1.0,1.0


In [21]:
# prospects turning around (compact)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] != -1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%52W', ascending=False)

qualified stocks: 4


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag
48,NATIONALUM.NS,83.0,3.31,-2.50,1.28,126.0,66.0,27.75,72.25,56.0,90.0,10.3,5.98,8.0,52.0,30.0,0.0,1.0
0,GODREJIND.NS,424.0,-5.94,-0.44,0.45,622.0,405.0,8.85,91.15,37.0,638.0,39.5,18.50,50.0,47.0,48.0,0.0,1.0
65,TCS.NS,3430.0,6.95,-0.37,0.49,3735.0,2915.0,62.77,37.23,67.0,3385.0,28.1,30.80,-1.0,9.0,4.0,1.0,0.0
66,HCLTECH.NS,1123.0,12.95,-0.10,0.37,1148.0,857.0,91.32,8.68,67.0,1117.0,16.3,21.10,-1.0,2.0,0.0,1.0,1.0


In [22]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] != -1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 1


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag
62,INDIGO.NS,2109.0,13.34,0.32,1.75,2265.0,1549.0,78.25,21.75,59.0,2176.0,30.7,1000.0,3.0,7.0,5.0,NaN,NaN


In [23]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] != -1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 5


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag
48,NATIONALUM.NS,83.0,3.31,-2.50,1.28,126.0,66.0,27.75,72.25,56.0,90.0,10.3,5.98,8.0,52.0,30.0,0.0,1.0
65,TCS.NS,3430.0,6.95,-0.37,0.49,3735.0,2915.0,62.77,37.23,67.0,3385.0,28.1,30.80,-1.0,9.0,4.0,1.0,0.0
0,GODREJIND.NS,424.0,-5.94,-0.44,0.45,622.0,405.0,8.85,91.15,37.0,638.0,39.5,18.50,50.0,47.0,48.0,0.0,1.0
66,HCLTECH.NS,1123.0,12.95,-0.10,0.37,1148.0,857.0,91.32,8.68,67.0,1117.0,16.3,21.10,-1.0,2.0,0.0,1.0,1.0
58,LTIM.NS,4431.0,-2.11,-2.03,-1.54,6390.0,3796.0,24.47,75.53,55.0,4663.0,26.6,37.90,5.0,44.0,24.0,1.0,0.0


In [24]:
# star stocks performance

star_stocks = ['HINDUNILVR.NS', 'BAJFINANCE.NS', 'EICHERMOT.NS', 'FEDERALBNK.NS',
          'CANBK.NS', 'TCS.NS', 'M&M.NS', 'LTIM.NS', 'ICICIBANK.NS',
          'AXISBANK.NS', 'HCLTECH.NS', 'TECHM.NS', 'WIPRO.NS',
          'INDUSINDBK.NS', 'JUBLFOOD.NS', 'DMART.NS', 'DABUR.NS',
          'TATAPOWER.NS', 'BHARTIARTL.NS']

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] != -1) & (tmp_df['DuraFlag'] != -1)]
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag
2,BAJFINANCE.NS,5796.0,-12.04,-0.99,-1.73,7720.0,5265.0,21.63,78.37,26.0,7869.0,50.8,35.40,36.0,33.0,34.0,1.0,1.0
12,JUBLFOOD.NS,494.0,-10.86,-0.36,-1.15,687.0,463.0,13.77,86.23,38.0,636.0,87.9,68.80,29.0,39.0,34.0,1.0,1.0
32,TATAPOWER.NS,201.0,-10.47,-0.89,-0.93,287.0,195.0,7.01,92.99,35.0,232.0,20.4,25.30,15.0,43.0,29.0,1.0,0.0
58,LTIM.NS,4431.0,-2.11,-2.03,-1.54,6390.0,3796.0,24.47,75.53,55.0,4663.0,26.6,37.90,5.0,44.0,24.0,1.0,0.0
37,DMART.NS,3515.0,-12.30,-0.55,-1.48,4577.0,3231.0,21.11,78.89,31.0,4006.0,138.7,97.10,14.0,30.0,22.0,1.0,1.0
21,EICHERMOT.NS,3214.0,2.42,1.15,-1.07,3850.0,2241.0,60.46,39.54,49.0,3916.0,40.1,37.70,22.0,20.0,21.0,1.0,1.0
16,ICICIBANK.NS,855.0,3.30,0.85,-0.44,953.0,650.0,67.76,32.24,36.0,1094.0,24.2,18.70,28.0,11.0,20.0,1.0,1.0
23,INDUSINDBK.NS,1156.0,9.28,1.23,0.50,1264.0,771.0,78.12,21.88,38.0,1402.0,22.8,13.20,21.0,9.0,15.0,1.0,1.0
26,AXISBANK.NS,892.0,13.82,1.02,0.65,962.0,625.0,79.33,20.67,36.0,1060.0,31.3,13.30,19.0,8.0,14.0,1.0,0.0
39,BHARTIARTL.NS,776.0,3.53,0.13,-0.74,851.0,638.0,65.02,34.98,44.0,879.0,NaN,72.10,13.0,10.0,12.0,1.0,1.0


In [25]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == -1) | (tmp_df['DuraFlag'] == -1)]
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag
4,IBREALEST.NS,71.0,-8.65,-1.28,-1.23,146.0,60.0,13.08,86.92,29.0,95.0,11.3,1000.00,34.0,106.0,70.0,-1.0,-1.0
3,INDUSTOWER.NS,158.0,-20.65,-0.50,-1.05,246.0,158.0,-0.11,100.11,17.0,213.0,NaN,15.50,35.0,56.0,46.0,-1.0,1.0
1,ZEEL.NS,220.0,-10.93,-1.21,-1.22,299.0,206.0,15.05,84.95,27.0,303.0,24.6,26.10,38.0,36.0,37.0,-1.0,1.0
5,NAM-INDIA.NS,242.0,-11.77,-1.46,-0.39,340.0,242.0,-0.26,100.26,30.0,322.0,29.9,22.50,33.0,40.0,36.0,-1.0,1.0
18,PVR.NS,1664.0,-8.25,-0.61,-0.91,2193.0,1498.0,23.93,76.07,45.0,2106.0,51.2,1000.00,27.0,32.0,30.0,1.0,-1.0
55,WIPRO.NS,402.0,-5.07,-2.36,0.00,605.0,374.0,11.93,88.07,57.0,425.0,19.0,19.40,6.0,50.0,28.0,-1.0,0.0
8,EMAMILTD.NS,420.0,-7.01,-0.22,-0.46,517.0,397.0,19.42,80.58,42.0,550.0,NaN,22.30,31.0,23.0,27.0,-1.0,1.0
24,DEEPAKNTR.NS,1837.0,-9.76,-0.83,-1.27,2356.0,1713.0,19.33,80.67,24.0,2231.0,28.4,27.30,21.0,28.0,24.0,-1.0,1.0
40,HAVELLS.NS,1180.0,-3.79,0.00,-0.42,1388.0,1065.0,35.76,64.24,52.0,1328.0,63.7,69.40,13.0,18.0,16.0,-1.0,1.0
57,HINDALCO.NS,489.0,15.35,-1.18,1.08,626.0,314.0,56.12,43.88,56.0,514.0,10.4,7.93,5.0,28.0,16.0,-1.0,1.0
